In [1]:
import json
import spacy
import tqdm
import pandas as pd
import re
import numpy as np
import os
import csv

aliases_list = [
    ["PMI", "philip morris"],
    ["FDA","food and drug", "fda","food & drug"],
    ["Andre Calantzopoulos","calantzopoulos"],
    ['IQOS', 'iqos'],
    ["JUUL","juul"],
    ["WHO","World Health Organisation"],
    ["the Royal College of Physicians","royal college of physicians"],
    ["CDC","centers for disease control"],
    ["ATHRA", "australian tobacco harm reduction association"],
    ["PHE", "public health england", "phe"]
] # load in alias list to account for synonyms: [["real name","alias1","alias2", ...], ...] accounts for synonyms

MIN_COUNT = 3  # MINIMUM NUMBER OF INSTANCES THAT A NAMED ENTITY MUST APPEAR
PRO_INDUSTRY_PREDICTION_THRESHOLD = .75 #PRO-INDUSTRY THRESHOLD WITH WHICH WE CONSIDER A DOCUMENT
CHANCE_TO_REPLACE_SNIPPET = 0 # set after feed loaded

PATH_TO_FEED = 'Files/feed.jsonl'
PATH_TO_CSV = "Files/corefs.csv"

BACKGROUND_FOREGROUND_ANALYSIS = True # Toggle background / foreground analysis
BACKGROUND_FOREGROUND_THRESHOLD = .35 # Threshold for ratio of foreground frequency to background frequency


nlp = spacy.load("en_core_web_lg") #natural language processing model
nlp.disable_pipes("tagger", "parser") #"parser" can't be disabled for spacy noun_chuncks

[('tagger', <spacy.pipeline.pipes.Tagger at 0x11b9dce80>),
 ('parser', <spacy.pipeline.pipes.DependencyParser at 0x11d5c7c48>)]

## Load Feed

In [2]:
examples = []  # list of articles
for line in open(PATH_TO_FEED):
    examples.append(json.loads(line))

CHANCE_TO_REPLACE_SNIPPET = 3.0 / len(examples)  # percent chance for a current snippet to replace a stored one
print("Loaded", len(examples), "documents")

FileNotFoundError: [Errno 2] No such file or directory: 'Files/feed.jsonl'

In [120]:
named_ents = {} #{"named entity": count} keeps count of each named entity ex: named_ents["PMI"] ->> 233 times
coreference_matrix = {} #{"named entity":{"coref1":count, "coref2":count}} keeps count of coreferenced named entities in a sentence ex: coreference_matrix["PMI"]["FDA"] ->>> 34 times
associated_keywords = {} #{"named entity": {"keyword1":count, "keyword2":count}} keeps count of keywords associated with named entities ex: associated_keywords["PMI"]["help"] ->>> 200 times
associated_countries = {} #{"named entity": {"country1":count, "country2":count}} keeps count of countries associated witg named entities ex: associated_countries["PMI"]["USA"] ->>> 20 times
snippets = {} # {"named entity": [snippet1, snippet2, snippet3]} stores example sentences of a named entity
background_ents = {} #{"named entity": count} keeps count of named entity regardless of whether the sentence is pro-industry


"""add_to_dict will update named_ents"""
def add_to_dict(ent_text): 
    if ent_text in named_ents:
        named_ents[ent_text]+=1
    else:
        named_ents[ent_text] = 1
        
"""add_to_matrix will update coreference_matrix"""
def add_to_matrix(ent1_text,ent2_text): 
    
    if ent1_text not in coreference_matrix:
        coreference_matrix[ent1_text] = {}
            
    if ent2_text not in coreference_matrix:
        coreference_matrix[ent2_text] = {}
    
    if ent1_text not in coreference_matrix[ent2_text]:
        coreference_matrix[ent2_text][ent1_text]=1
        coreference_matrix[ent1_text][ent2_text]=1
    else:
        coreference_matrix[ent2_text][ent1_text]+=1 
        if not ent2_text == ent1_text: #add it only once if label1 == label2
            coreference_matrix[ent1_text][ent2_text]+=1
    
"""add_to_keywords will update associated_keywords"""
def add_to_keywords(ent_text, keyword):
    if not ent_text in associated_keywords:
        associated_keywords[ent_text] = {keyword: 1}
    elif not keyword in associated_keywords[ent_text]:
        associated_keywords[ent_text][keyword] = 1 
    else:
        associated_keywords[ent_text][keyword] += 1
        
"""add_to_countries will update associated_countries"""
def add_to_countries(ent_text, countries):
    if not ent_text in associated_countries:
        associated_countries[ent_text] = {countries: 1}
    elif not countries in associated_countries[ent_text]:
        associated_countries[ent_text][countries] = 1 
    else:
        associated_countries[ent_text][countries] += 1

"""add_to_snippets will update associated_keywords"""
def add_to_snippets(ent_text, snippet):
    if not ent_text in snippets:
        snippets[ent_text] = [snippet]
    elif snippet in snippets[ent_text]:
        return
    elif len(snippets[ent_text]) < 3:
        snippets[ent_text].append(snippet)
    else:
        if np.random.random() < CHANCE_TO_REPLACE_SNIPPET:
            snippets[int(np.random.randint(0,high=2))] = snippet
        
def add_to_background_ents(ent_text):
    if ent_text in background_ents:
        background_ents[ent_text]+=1
    else:
        background_ents[ent_text]=1
        
"""check_alias will check if ent_text is alias, if so it will return the proper name"""
def check_alias(ent_text):
    for aliases in aliases_list:
        for alias in aliases[1:]:
            if alias in ent_text.lower():
                return aliases[0]
    return ent_text

## Iterate Over Data

In [121]:
# iterate over every article
for example in tqdm.tqdm_notebook(examples):
    
    # iterate over every sentence
    for sent, prob in zip(example["_source"]["sents"], example["_source"]["POSITIVE_sent_probas"]):
            
        # Condition for BACKGROUND_FOREGROUND_ANALYSIS boolean
        if BACKGROUND_FOREGROUND_ANALYSIS or prob > PRO_INDUSTRY_PREDICTION_THRESHOLD: 
            doc = nlp(sent)
            ents = list(doc.ents) 
            
            # iterate through entities
            for i in range(len(ents)):
                if ents[i].label_ == "PERSON" or ents[i].label_ == "ORG":

                    # check alias:
                    ent1_text = ents[i].text
                    ent1_text = re.sub("[\n,]", "", ent1_text)
                    ent1_text = check_alias(ent1_text)

                    #check incomplete name:
                    if ents[i].label_== "PERSON" and not " " in ent1_text and ent1_text != "JUUL": #JUUL IS MISTAKENLY THOUGHT TO BE A PERSON
                        break



                    if prob > PRO_INDUSTRY_PREDICTION_THRESHOLD: # if model is > 75% confident that the sentence is pro-industry
                        # update named_ents
                        add_to_dict(ent1_text)

                        # update keywords
                        for token in doc:#noun_chunk in doc.noun_chunks:
                            if (not token.is_stop) and token.is_alpha and (not token.text.lower() in ent1_text.lower()):
                                add_to_keywords(ent1_text, token.lemma_.lower())
                            #add_to_keywords(ent1_text, noun_chunk.text.lower())

                        # update countires
                        for country in example["_source"]["countries"]:
                            add_to_countries(ent1_text, country["country_name"])

                        # update snippets
                        add_to_snippets(ent1_text, re.sub('"', '""', re.sub("\n", "", doc.text)))

                        # iterate corefs
                        for j in range(len(ents) - i):
                            if ents[j + i].label_ == "PERSON" or ents[j + i].label_ == "ORG":
                                # check alias:
                                ent2_text = ents[j + i].text
                                ent3_text = re.sub("[\n,]", "", ent2_text)
                                ent2_text = check_alias(ent2_text)

                                # update coreference_matrix
                                add_to_matrix(ent1_text, ent2_text)

                    # update background_ents - regardless of pro-industry statement
                    if BACKGROUND_FOREGROUND_ANALYSIS:
                        add_to_background_ents(ent1_text)


HBox(children=(IntProgress(value=0, max=18308), HTML(value='')))

KeyboardInterrupt: 

## Clean Data

In [ ]:
# discard any named entities that are below the minimum count threshold from named_ents and associated_keywords
# NOTE: only partially deletes from coreference matrix. not accounted for: coref["kept": {"kept": 90, "DISCARD":1}]
# This is handled later

# REMOVE ENTITIES BELOW THRESHOLD OR IF THEY HAVE A RELATIVELY HIGH BACKGROUND FREQUENCY:
keys = list(named_ents.keys())
for i in range(len(named_ents)):
    if named_ents[keys[i]] < MIN_COUNT or (BACKGROUND_FOREGROUND_ANALYSIS and named_ents[keys[i]]/background_ents[keys[i]] < BACKGROUND_FOREGROUND_THRESHOLD):
        del named_ents[keys[i]]


# REMOVE UNWANTED ENTITIES
remove_ents = ["FDA","CDC","Vaping","Vapes","E-Cigarette","Nicotine","WHO"]

for ent in remove_ents:
    if ent in named_ents:
        del named_ents[ent]

# Sort Data

In [ ]:
sorted_named_ents = sorted(named_ents.items(), key=lambda kv: kv[1], reverse=True)

## Export to CSV

In [ ]:
# FORMAT:
# entity, count, snippets, associated keywords, coreferences
with open(PATH_TO_CSV, "w+") as f:

    for i, row in enumerate(sorted_named_ents):
        ent_text = row[0]
        #print(ent_text, named_ents[ent_text],background_ents[ent_text], named_ents[ent_text]/background_ents[ent_text])

        # entity
        f.write(ent_text)
        f.write(',')

        # count
        f.write(str(named_ents[ent_text]))
        f.write(',')

        # snippets
        f.write('"')
        f.write(snippets[ent_text][0])
        f.write('","')
        f.write(snippets[ent_text][1] if len(snippets[ent_text])>1 else "")
        f.write('","')
        f.write(snippets[ent_text][2] if len(snippets[ent_text])>2 else "")
        f.write('",')

        # associated keywords
        MIN_KEYWORD_COUNT = 2
        f.write('"')
        if ent_text in associated_keywords:
            sorted_associated_keywords = sorted(associated_keywords[ent_text].items(), key=lambda kv: kv[1], reverse=True)
            for row in sorted_associated_keywords:
                keyword = row[0]
                if associated_keywords[ent_text][keyword] > MIN_KEYWORD_COUNT:
                    f.write(keyword)
                    f.write(':')
                    f.write(str(associated_keywords[ent_text][keyword]))
                    f.write(', ')
        f.write('",')

        # coreferences
        f.write('"')
        sorted_corefs = sorted(coreference_matrix[ent_text].items(), key=lambda kv: kv[1], reverse= True)
        for row in sorted_corefs:
            coref = row[0]
            if coref in named_ents:  # accounting for incomplete deletion in coreference_matrix
                f.write(coref)
                f.write(':')
                f.write(str(coreference_matrix[ent_text][coref]))
                f.write(', ')
        f.write('",')

        # COUNRIES
        f.write('"')
        if ent_text in associated_countries:
            sorted_associated_countries = sorted(associated_countries[ent_text].items(), key=lambda kv: kv[1], reverse= True)
            for row in sorted_associated_countries:
                country = row[0]
                f.write(country)
                f.write(':')
                f.write(str(associated_countries[ent_text][country]))
                f.write(', ')
        f.write('"')

        f.write('\n')

In [55]:
#TODO: noun_chuncks
#TODO: deduping
#TODO: background / forground - only use Tokenizer

# Networkx

In [95]:
import networkx as nx

In [ ]:
graph = nx.Graph()
for ent, val in named_ents.items():
    graph.add_node(ent,weight=val)
for ent, corefs in coreference_matrix.items():
    for coref, link_weight in corefs.items():
        if coref in named_ents and ent in named_ents:
            graph.add_edge(ent,coref,weight=link_weight)

In [ ]:
nx.write_graphml(graph,"/Users/Ben/Desktop/Vital Strategies/networkxgraphs/feedtest3.graphml")